In [ ]:
include("_.jl")

In [ ]:
using XMLDict

In [ ]:
ca_se_di = Dict()

en = "GENESET"

for li in readlines(joinpath(PAI, "msigdb_v7.5.1.xml"))

    if !startswith(li, "<$en")

        continue

    end

    di = xml_dict(li)[en]

    ca = di[:CATEGORY_CODE]

    if ca == "ARCHIVED"

        continue

    end

    if !haskey(ca_se_di, ca)

        ca_se_di[ca] = Dict()

    end

    ca_se_di[ca][di[:STANDARD_NAME]] = Dict(
        "Organism" => di[:ORGANISM],
        "Description" => di[:DESCRIPTION_BRIEF],
        "URL" => di[:EXTERNAL_DETAILS_URL],
        "ENSGs" => split(di[:MEMBERS], ","),
        "Symbols" => split(di[:MEMBERS_SYMBOLIZED], ","),
    )

end

In [ ]:
co = joinpath(PAO, "collection")

mkpath(co)

In [ ]:
for (ca, se_di) in ca_se_di

    println("$ca has $(length(se_di)) sets.")

    js = "$(lowercase(ca)).json"

    OnePiece.dict.write(joinpath(PAO, js), se_di)

    OnePiece.dict.write(joinpath(co, js), Dict(se => di["Symbols"] for (se, di) in se_di))

end

In [ ]:
for (ro, di_, fi_) in walkdir(PAI)

    for fi in fi_

        pr, ex = splitext(fi)

        if ex in (".gmt", ".tsv")

            se_ge_ = OnePiece.gmt.read(joinpath(ro, fi))

            println("$pr has $(length(se_ge_)) sets.")

            OnePiece.dict.write(joinpath(co, "$pr.json"), se_ge_)

        end

    end

end